In [1]:
import os
import time
import Turbine
import mat_properties as prop
import numpy as n
import pandas as pd

os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,508.600000,5.1990,3414.093682,63.40,NaN,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN,NaN
DROSND-TURBND,209.800000,0.4000,2859.669112,14.76,NaN,NaN
SMESHEND,197.800000,0.5490,2848.521603,78.16,NaN,NaN
DOOTB2,131.400000,0.2542,2724.861533,78.16,NaN,NaN
DOOTB1,NaN,0.1525,2652.000000,78.16,NaN,NaN
INCND,NaN,0.1525,2652.000000,78.16,NaN,NaN
INKOND,41.510000,0.0017,2338.024560,78.16,NaN,NaN


In [2]:
calctolerance = 10**-3
calcmethod = "hybr"
KPD_SP = 0.99  # эффективность сетевых подогревателей


Turb = Turbine.turbine(
    water,
    "DROSVD-TURBVD",
    "ENDOFVD",
    "DROSND-TURBND",
    "SMESHEND",
    "DOOTB2",
    "DOOTB1",
    "INCND",
    "INKOND",
    water_streams0,
    water_streams,
)

start_time = time.time()

res = Turb.calculate(diafragma=0.2, maxiterations=50, calctolerance=10**-5)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))
res

fin:--- 0.1 сек. ---


[0.8462130543785595,
 0.8207056532332418,
 0.7719792559731901,
 0.7926277968842296]

In [3]:
# надо посчитать потерю давления НД (готово)
# отборы неправильно (второй отбор завышен, надопроверить исходные данные функции)
water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,505.102827,7.830656,3414.093682,63.40,6.754673,100.00000
ENDOFVD,207.638637,0.645437,2865.023235,63.40,6.965940,100.00000
DROSND-TURBND,206.628540,0.707437,2859.669112,14.76,6.913922,100.00000
SMESHEND,207.177334,0.645437,2864.012144,78.16,6.963836,100.00000
DOOTB2,167.468217,0.419504,2790.215862,78.16,6.996010,100.00000
DOOTB1,155.599204,0.362076,2768.419081,78.16,7.011747,100.00000
INCND,148.245618,0.162076,2768.419081,78.16,7.374933,100.00000
INKOND,54.232740,0.015192,2477.466912,78.16,7.632206,94.88637


In [4]:
water_streams0['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S
DROSVD-TURBVD,508.600000,7.8450,3422.525472,63.40,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN
DROSND-TURBND,209.800000,0.6110,2871.426945,14.76,7.003739
SMESHEND,197.800000,0.5490,2848.521603,78.16,7.003601
DOOTB2,131.400000,0.2542,2724.861533,78.16,7.065770
DOOTB1,NaN,0.1525,2652.000000,78.16,7.109000
INCND,NaN,0.1525,2652.000000,78.16,NaN
INKOND,41.510000,0.0080,2341.254319,78.16,NaN


In [5]:
Get_value='P'
Error=(water_streams['DROSVD-TURBVD':'INKOND'][Get_value]-water_streams0['DROSVD-TURBVD':'INKOND'][Get_value])/water_streams0['DROSVD-TURBVD':'INKOND'][Get_value]*100
Error

DROSVD-TURBVD     -0.182847
ENDOFVD           17.565920
DROSND-TURBND     15.783453
SMESHEND          17.565920
DOOTB2            65.029128
DOOTB1           137.426829
INCND              6.279288
INKOND            89.896480
Name: P, dtype: float64

In [6]:
Turb.calculate_power()

{'Ni': 65.02344170733772,
 'Ni1': 34.8110663564488,
 'Ni2': 5.767917454407764,
 'Ni3': 22.740821528760073,
 'Ni4': 1.7036363677210897}